In [1]:
import scipy.sparse as sparse
import scipy.io as sio
import scipy.stats as stats
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
from time import time
# own code
import graph_tools as gtl
import invarients as inva
import cpp

In [2]:
G = gtl.read_graph_from_edgelist("networks-random/brain274/bn-mouse_brain_1.edges")


In [3]:
start = time()
inva.q_extent_single(3,G)
end = time()
clock = end-start
print(clock)

In [8]:
start = time()


dists = gtl.convert_dist_dict_to_narray(dict(nx.all_pairs_dijkstra_path_length(G)),list(G.nodes))
# cpp.q_extend_single()

end = time()
clock  = end -start
print(clock)

cpp.q_extend_single(dists, 3)

0.5468525886535645
